In [1]:
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingRegressor

### Обучение и выполнение лучшей модели

In [3]:
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения (тюник в сопровождающих файлах)

In [4]:
# Путь к тренировочному набору
path_train =  'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  =  'test.csv'# содержит только имя файла, без имен папок

In [5]:
# Блок(и) обучения и поверки модели, а также отбора необходимых значимых столбцов

In [6]:
data_train = pd.read_csv(path_train)
data_test = pd.read_csv(path_test)

In [7]:
data_train

,company,model,year,condition,mileage(km),fuel,volume(cm3),color,transmission,drive_unit,vehicle_size_class,priceUSD
0,renault,laguna,2000,with mileage,230000.0,petrol,1800.0,blue,mechanics,front-wheel drive,D,2300
1,chrysler,pacifica,2005,with mileage,310000.0,petrol,3500.0,black,auto,front-wheel drive,NaN,6300
2,mercedes-benz,c-klass,2010,with mileage,115000.0,petrol,1800.0,gray,auto,rear drive,D,11200
3,hyundai,accent,2017,with mileage,45500.0,petrol,1400.0,gray,auto,front-wheel drive,B,10886
4,renault,kadjar,2016,with mileage,139000.0,diesel,1500.0,red,auto,front-wheel drive,J,15300
...,...,...,...,...,...,...,...,...,...,...,...,...
38000,renault,laguna,1999,with mileage,314000.0,petrol,1600.0,green,mechanics,front-wheel drive,D,1999
38001,opel,insignia,2010,with mileage,230000.0,petrol,1600.0,black,mechanics,front-wheel drive,D,7606
38002,kia,optima,2012,with mileage,115000.0,petrol,2400.0,black,mechanics,front-wheel drive,D,11850
38003,volkswagen,vento,1994,with mileage,300000.0,diesel,1900.0,white,mechanics,front-wheel drive,NaN,1200


In [10]:
data_train = data_train.drop_duplicates()
data_train

,company,model,year,condition,mileage(km),fuel,volume(cm3),color,transmission,drive_unit,vehicle_size_class,priceUSD
0,renault,laguna,2000,with mileage,230000.0,petrol,1800.0,blue,mechanics,front-wheel drive,D,2300
1,chrysler,pacifica,2005,with mileage,310000.0,petrol,3500.0,black,auto,front-wheel drive,NaN,6300
2,mercedes-benz,c-klass,2010,with mileage,115000.0,petrol,1800.0,gray,auto,rear drive,D,11200
3,hyundai,accent,2017,with mileage,45500.0,petrol,1400.0,gray,auto,front-wheel drive,B,10886
4,renault,kadjar,2016,with mileage,139000.0,diesel,1500.0,red,auto,front-wheel drive,J,15300
...,...,...,...,...,...,...,...,...,...,...,...,...
38000,renault,laguna,1999,with mileage,314000.0,petrol,1600.0,green,mechanics,front-wheel drive,D,1999
38001,opel,insignia,2010,with mileage,230000.0,petrol,1600.0,black,mechanics,front-wheel drive,D,7606
38002,kia,optima,2012,with mileage,115000.0,petrol,2400.0,black,mechanics,front-wheel drive,D,11850
38003,volkswagen,vento,1994,with mileage,300000.0,diesel,1900.0,white,mechanics,front-wheel drive,NaN,1200


In [11]:
cat_features = ['company', 'model', 'condition', 'fuel', 'color', 'transmission', 'drive_unit' , 'vehicle_size_class']
num_features = ['year', 'mileage(km)', 'volume(cm3)']

In [12]:
X_train = data_train.drop(columns = 'priceUSD')
y_train = data_train['priceUSD']

In [13]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')), 
    ('onehot', OneHotEncoder(handle_unknown='ignore' ,  sparse_output=False))
])

In [14]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

In [15]:
CT = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_pipeline, cat_features)
    ] , remainder='drop').set_output(transform='pandas')

In [16]:
ct = CT.fit_transform(X_train)

In [17]:
ct

,num__year,num__mileage(km),num__volume(cm3),cat__company_acura,cat__company_alfa-romeo,cat__company_aro,cat__company_asia,cat__company_aston-martin,cat__company_audi,cat__company_bentley,...,cat__vehicle_size_class_A,cat__vehicle_size_class_B,cat__vehicle_size_class_C,cat__vehicle_size_class_D,cat__vehicle_size_class_E,cat__vehicle_size_class_F,cat__vehicle_size_class_J,cat__vehicle_size_class_M,cat__vehicle_size_class_S,cat__vehicle_size_class_Unknown
0,-0.333333,0.000000,-0.281429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.083333,0.465116,2.147143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.500000,-0.668605,-0.281429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.083333,-1.072674,-0.852857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,-0.529070,-0.710000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38000,-0.416667,0.488372,-0.567143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38001,0.500000,0.000000,-0.567143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38002,0.666667,-0.668605,0.575714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38003,-0.833333,0.406977,-0.138571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
model_pipe_rf = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', RandomForestRegressor(n_estimators=1000 , max_depth = None , max_features = 1.0 ,min_samples_leaf = 1 ,min_samples_split=5))
                            ]),
                        transformer=PowerTransformer(),
                        check_inverse = False)
                )])
model_pipe_rf

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(strategy='median')),
                                                                                                                        ('scaler',
                                                                                                                         RobustScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(fill_value='Unknown',
                                                                                                                                       strategy='constant')),
                                                                                                                        ('onehot',
                                                                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                                                                       sparse_output=False))]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'color',
                                                                                                         'transmission',
                                                                                                         'drive_unit',
                                                                                                         'vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       RandomForestRegressor(min_samples_split=5,
                                                                                             n_estimators=1000))]),
                                            transformer=PowerTransformer()))])

In [20]:
model_pipe_gb = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', GradientBoostingRegressor(n_estimators=1000 , learning_rate = 0.2 , max_depth = 5 , min_samples_leaf = 2 , min_samples_split = 2))
                            ]),
                        transformer=PowerTransformer(),
                        check_inverse = False)
                )])
model_pipe_gb

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(strategy='median')),
                                                                                                                        ('scaler',
                                                                                                                         RobustScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(fill_value='Unknown',
                                                                                                                                       strategy='constant')),
                                                                                                                        ('onehot',
                                                                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                                                                       sparse_output=False))]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'color',
                                                                                                         'transmission',
                                                                                                         'drive_unit',
                                                                                                         'vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       GradientBoostingRegressor(learning_rate=0.2,
                                                                                                 max_depth=5,
                                                                                                 min_samples_leaf=2,
                                                                                                 n_estimators=1000))]),
                                            transformer=PowerTransformer()))])

In [21]:
model_pipe_svr = Pipeline([
    ('model', TransformedTargetRegressor(
            regressor=Pipeline(steps=[
                ('preproc', CT),
                ('estimator', SVR())
            ]),
            transformer=PowerTransformer(),
            check_inverse=False)
    )
])
model_pipe_svr

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(strategy='median')),
                                                                                                                        ('scaler',
                                                                                                                         RobustScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('imputer',
                                                                                                                         SimpleImputer(fill_value='Unknown',
                                                                                                                                       strategy='constant')),
                                                                                                                        ('onehot',
                                                                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                                                                       sparse_output=False))]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'color',
                                                                                                         'transmission',
                                                                                                         'drive_unit',
                                                                                                         'vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       SVR())]),
                                            transformer=PowerTransformer()))])

In [22]:
voiting = VotingRegressor([('rf', model_pipe_rf), ('gb', model_pipe_gb), ('svr', model_pipe_svr)])

In [23]:
voiting.fit(X_train, y_train)

VotingRegressor(estimators=[('rf',
                             Pipeline(steps=[('model',
                                              TransformedTargetRegressor(check_inverse=False,
                                                                         regressor=Pipeline(steps=[('preproc',
                                                                                                    ColumnTransformer(transformers=[('num',
                                                                                                                                     Pipeline(steps=[('imputer',
                                                                                                                                                      SimpleImputer(strategy='median')),
                                                                                                                                                     ('scaler',
                                                                                                                                                      RobustScaler())]),
                                                                                                                                     ['year',
                                                                                                                                      'mileage(km)',
                                                                                                                                      'volume(cm3)']),
                                                                                                                                    ('cat',
                                                                                                                                     Pipeline(steps=[('imputer',
                                                                                                                                                      SimpleI...
                                                                                                                                      'mileage(km)',
                                                                                                                                      'volume(cm3)']),
                                                                                                                                    ('cat',
                                                                                                                                     Pipeline(steps=[('imputer',
                                                                                                                                                      SimpleImputer(fill_value='Unknown',
                                                                                                                                                                    strategy='constant')),
                                                                                                                                                     ('onehot',
                                                                                                                                                      OneHotEncoder(handle_unknown='ignore',
                                                                                                                                                                    sparse_output=False))]),
                                                                                                                                     ['company',
                                                                                                                                      'model',
                                                                                                                                      'condition',
                                       

In [24]:
y_predict = voiting.predict(data_test)

In [28]:
y_predict

array([10543.19248146,  8318.9337078 ,  2582.07064143, ...,
       10277.13689558,  4806.1873706 ,  9001.72459287])

In [29]:
#negative_values = y_predict[y_predict < 0] отрицательных цен нет